- Requires `tiktoken` tokenizer package.

```sh
        pip install -q tiktoken
```

In [1]:
import torch
import tiktoken

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,  # Embedding dimension
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,  # Number of layers
    "drop_rate": 0.1,  # Dropout rate
    "qkv_bias": False,  # Query-Key-Value bias
}

### A placeholder GPT model architecture class(es)

In [3]:
class DummyGPTModel(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = torch.nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = torch.nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = torch.nn.Dropout(cfg["drop_rate"])
        self.transformer_blocks = torch.nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = torch.nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x


class DummyLayerNorm(torch.nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
        return x

In [4]:
text_1 = "Every effort moves you"
text_2 = "Every day holds a"

tokenizer = tiktoken.get_encoding("gpt2")

batch = []
batch.append(torch.tensor(tokenizer.encode(text_1)))
batch.append(torch.tensor(tokenizer.encode(text_2)))

batch = torch.stack(batch, dim=0)
print(batch.shape)

torch.Size([2, 4])


In [5]:
torch.manual_seed(123)

model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print(logits.shape)

print(logits)

torch.Size([2, 4, 50257])
tensor([[[-1.2034,  0.3201, -0.7130,  ..., -1.5548, -0.2390, -0.4667],
         [-0.1192,  0.4539, -0.4432,  ...,  0.2392,  1.3469,  1.2430],
         [ 0.5307,  1.6720, -0.4695,  ...,  1.1966,  0.0111,  0.5835],
         [ 0.0139,  1.6754, -0.3388,  ...,  1.1586, -0.0435, -1.0400]],

        [[-1.0908,  0.1798, -0.9484,  ..., -1.6047,  0.2439, -0.4530],
         [-0.7860,  0.5581, -0.0610,  ...,  0.4835, -0.0077,  1.6621],
         [ 0.3567,  1.2698, -0.6398,  ..., -0.0162, -0.1296,  0.3717],
         [-0.2407, -0.7349, -0.5102,  ...,  2.0057, -0.3694,  0.1814]]],
       grad_fn=<UnsafeViewBackward0>)


### Layer normalization class

In [6]:
class LayerNorm(torch.nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = torch.nn.Parameter(torch.ones(emb_dim))
        self.shift = torch.nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

### GELU Activation

In [7]:
class GELU(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2. / torch.pi)) * (x + 0.044715 * torch.pow(x, 3))))

### GELU based feed forward layer

In [8]:
class FeedForwardGELU(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(cfg["emb_dim"], cfg["emb_dim"] * 4), # manipulate the embedding dimension internally.
            GELU(), # Non-linear activation on expanded input.
            torch.nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), # restore the original embedding dimension.
        )

    def forward(self, x):
        return self.layers(x)

### Putting it all together in a Transformer block

In [9]:
from SelfAttention import MultiHeadAttentionWrapper, MultiHeadAttention

In [10]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layernorm_1 = LayerNorm(cfg["emb_dim"])
        self.mha = MultiHeadAttention(
            dim_in=cfg["emb_dim"],
            dim_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            scale=True,
            qkv_bias=cfg["qkv_bias"],
        )

        self.dropout_1 = torch.nn.Dropout(cfg["drop_rate"])
        self.layernorm_2 = LayerNorm(cfg["emb_dim"])
        self.ffn = FeedForwardGELU(cfg)
        self.dropout_2 = torch.nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        x = self.layernorm_1(x)
        x = self.mha(x)
        x = self.dropout_1(x)
        x = x + shortcut

        shortcut = x
        x = self.layernorm_2(x)
        x = self.ffn(x)
        x = self.dropout_2(x)
        x = x + shortcut
        return x

### Putting it all together - GPT model architecture implementation

In [11]:
class VanillaGPT(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.token_emb = torch.nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = torch.nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = torch.nn.Dropout(cfg["drop_rate"])

        self.transformer_blocks = torch.nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = torch.nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.token_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))

        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [12]:
torch.manual_seed(123)
model = VanillaGPT(GPT_CONFIG_124M)
logits = model(batch)

print(f"{batch=}")
print(f"{logits.shape=}")
print(f"{logits=}")

batch=tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
logits.shape=torch.Size([2, 4, 50257])
logits=tensor([[[ 0.3613,  0.4222, -0.0711,  ...,  0.3483,  0.4661, -0.2838],
         [-0.1792, -0.5660, -0.9485,  ...,  0.0477,  0.5181, -0.3168],
         [ 0.7120,  0.0332,  0.1085,  ...,  0.1018, -0.4327, -0.2553],
         [-1.0076,  0.3418, -0.1190,  ...,  0.7195,  0.4023,  0.0532]],

        [[-0.2564,  0.0900,  0.0335,  ...,  0.2659,  0.4454, -0.6806],
         [ 0.1230,  0.3653, -0.2074,  ...,  0.7705,  0.2710,  0.2246],
         [ 1.0558,  1.0318, -0.2800,  ...,  0.6936,  0.3205, -0.3178],
         [-0.1565,  0.3926,  0.3288,  ...,  1.2630, -0.1858,  0.0388]]],
       grad_fn=<UnsafeViewBackward0>)


### Example

In [13]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]  # limits to context_size tokens
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]  # latest time step only
        probas = torch.softmax(logits, dim=-1)  # dimension: (batch_size, vocab_size)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # dimension: (batch_size, 1)
        idx = torch.cat((idx, idx_next), dim=1)  # append to the existing sequence

    return idx

In [14]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)    # batch dimension
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [ ]:
model.eval()                  # switch to evaluation mode
out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))